In [4]:
import pandas as pd

In [5]:
simple_train=['call you tonight', 'Call me a cab', 'please call me... PLEASE!']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vect= CountVectorizer()

In [7]:
vect.fit(simple_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [8]:
vect.get_feature_names()

['cab', 'call', 'me', 'please', 'tonight', 'you']

In [81]:
#transform training data into a 'document-term matrix'
simple_train_dtm=vect.transform(simple_train)
simple_train_dtm

<3x7453 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [82]:
# convert sparse matrix to a dense matrix
simple_train_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


In [12]:
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [13]:
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


In [14]:
simple_test=["please don't call me"]

In [15]:
simple_test_dtm=vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]])

In [16]:
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


In [17]:
sms=pd.read_table('https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv', header=None, names=['label', 'message'])
sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
sms.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [19]:
sms['label_num']=sms.label.map({'ham':0, 'spam':1})

In [20]:
sms.head()

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [21]:
X=sms.message
y=sms.label_num

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


In [23]:
vect= CountVectorizer()

In [24]:
vect.fit(X_train)
X_train_dtm=vect.transform(X_train)

In [25]:
X_train_dtm

<4179x7453 sparse matrix of type '<class 'numpy.int64'>'
	with 55462 stored elements in Compressed Sparse Row format>

In [26]:
X_test_dtm=vect.transform(X_test)
X_test_dtm

<1393x7453 sparse matrix of type '<class 'numpy.int64'>'
	with 17360 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

In [28]:
nb.fit(X_train_dtm,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [29]:
y_pred_class=nb.predict(X_test_dtm)

In [31]:
from sklearn import metrics
metrics.accuracy_score(y_test,y_pred_class)

0.9856424982053122

In [32]:
metrics.confusion_matrix(y_test,y_pred_class)

array([[1188,    4],
       [  16,  185]])

In [83]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob=nb.predict_proba(X_test_dtm)[:,1]
y_pred_prob

array([1.52242828e-07, 6.20660265e-04, 1.00000000e+00, ...,
       1.69937721e-04, 1.12728116e-05, 5.18438883e-11])

In [35]:
metrics.roc_auc_score(y_test,y_pred_class)

0.958521152626131

In [37]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()

In [38]:
logreg.fit(X_train_dtm,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [84]:
# store the vocabulary of X_train
X_train_tokens=vect.get_feature_names()
len(X_train_tokens)

7453

In [41]:
print(X_train_tokens[0:50])
print(X_train_tokens[-50:])

['00', '000', '008704050406', '0089', '0121', '01223585334', '0125698789', '02', '0207', '02073162414', '02085076972', '021', '03', '04', '05', '050703', '0578', '06', '07', '07046744435', '07090201529', '07090298926', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '0776xxxxxxx', '07781482378', '07786200117', '077xxx', '07801543489', '07808247860', '07808726822', '07815296484', '07821230901', '078498', '07880867867', '0789xxxxxxx', '07946746291', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767']
['yi', 'yifeng', 'yijue', 'ym', 'ymca', 'yo', 'yoga', 'yogasana', 'yor', 'yorge', 'you', 'youdoing', 'young', 'younger', 'youphone', 'your', 'youre', 'yourjob', 'yours', 'yourself', 'youuuuu', 'youwanna', 'yoville', 'yowifes', 'yoyyooo', 'yr', 'yrs', 'ystrday', 'ything', 'yummmm', 'yummy', 'yun', 'yunny', 'yuo', 'yuou', 'yup', 'yupz', 'zaher', 'zealand', 'zed', 'zeros', 'zhong', 'zindgi', 

In [85]:
# Naive Bayes counts the number of times each token appears in each class
nb.feature_count_

array([[ 0.,  0.,  0., ...,  1.,  1.,  1.],
       [ 9., 23.,  1., ...,  0.,  0.,  0.]])

In [44]:
nb.feature_count_.shape

(2, 7453)

In [86]:
# number of times each token appears across all HAM messages
ham_token_count=nb.feature_count_[0,:]
ham_token_count

array([0., 0., 0., ..., 1., 1., 1.])

In [69]:
spam_token_count=nb.feature_count_[1,:]
spam_token_count

array([ 9., 23.,  1., ...,  0.,  0.,  0.])

In [70]:
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count, 'spam':spam_token_count}).set_index('token')
tokens.head()

,ham,spam
token,,
00,0.0,9.0
000,0.0,23.0
008704050406,0.0,1.0
0089,0.0,1.0
0121,0.0,1.0


In [71]:
tokens.sample(5)

,ham,spam
token,,
heard,8.0,1.0
87121,0.0,4.0
makiing,1.0,0.0
block,1.0,1.0
snap,0.0,1.0


In [87]:
nb.class_count_  # Naive Bayes counts the number of observations in each class

array([3633.,  546.])

In [89]:
tokens['ham']=tokens['ham']+1     # add 1 to ham and spam counts to avoid dividing by 0
tokens['spam']=tokens['spam']+1

tokens['ham'] = tokens.ham / nb.class_count_[0]    # convert the ham and spam counts into frequencies
tokens['spam'] = tokens.spam / nb.class_count_[1]
tokens.sample(5)

,ham,spam,spam ratio
token,,,
desires,0.000275,0.001835,6.664198
bleak,0.000275,0.001835,6.664198
registered,0.000275,0.001835,6.664241
icic,0.000275,0.001835,6.664198
start,0.000275,0.001835,6.664320


In [78]:
tokens['spam ratio']=tokens.spam/tokens.ham
tokens.sample(5)

,ham,spam,spam ratio
token,,,
course,0.000275,0.001835,6.664194
09095350301,0.000275,0.001835,6.664221
eightish,0.000275,0.001835,6.664198
amla,0.000275,0.001835,6.664198
discuss,0.000275,0.001835,6.664196


In [80]:
tokens.sort_values('spam ratio',ascending=False)

,ham,spam,spam ratio
token,,,
to,0.000275,0.001835,6.675278
call,0.000275,0.001835,6.669759
your,0.000275,0.001835,6.668121
you,0.000275,0.001835,6.668064
free,0.000275,0.001835,6.667790
for,0.000275,0.001835,6.667402
now,0.000275,0.001835,6.667387
the,0.000275,0.001835,6.667287
or,0.000275,0.001835,6.667189
